In [ ]:
!pip install flash_attn==2.5.8
!pip install torch==2.3.1
!pip install accelerate==0.31.0
!pip install transformers==4.41.2
!pip install datasets
!pip install transformers
!pip install trl
!pip install peft 
!pip install auto-gptq 
!pip install optimum
!pip install xformers
!pip install huggingface_hub
!pip install git+https://github.com/microsoft/LoRA
    
#bits and bytes with cuda
!pip install bitsandbytes-cuda110 bitsandbytes

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 38.9 MB/s eta 0:00:00a 0:00:01
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.2/43.2 kB 2.5 MB/s eta 0:00:00
  Created wheel for flash_attn: filename=flash_attn-2.5.8-cp310-cp310-linux_x86_64.whl size=120607435 sha256=b962f0eb38a2e54a3ece20b4b43f59f5a638ce53fe6e269992c58b119425d1f0
  Stored in directory: /root/.cache/pip/wheels/9b/5b/2b/dea8af4e954161c49ef1941938afcd91bb93689371ed12a226
Successfully built flash_attn
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 779.1/779.1 MB 1.2 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 2.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 83.3 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 69.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 39.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

### Important: You must restart the kernel at this point after installing the packages!!
---

# Preparing datasets, loading model and tokenizer, Training model 
## Model used: microsoft/Phi-3-mini-4k-instruct

In [ ]:
#load tokens
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
hf_token = user_secrets.get_secret("HF_TOKEN")

#logging into Hugging Face
!huggingface-cli login --token $hf_token

In [ ]:
# impoting classes
from random import randrange

import torch
from datasets import load_dataset

from peft import LoraConfig, prepare_model_for_kbit_training, PeftModel
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments,
    set_seed,
    pipeline
)
from trl import SFTTrainer

In [ ]:
# Preparing datasets

# DATASET_NAME is a string that specifies the name of the dataset to be used for fine-tuning.
DATASET_NAME = synthetic_text_to_sql_dataset_name = "gretelai/synthetic_text_to_sql"

# Load the dataset specified by DATASET_NAME using the load_dataset function.
dataset = load_dataset(DATASET_NAME)

dataset

# Extract relevant fields

# old
# def extract_fields_synthetic(example):
#     return {
#         "question": example["sql_prompt"],
#         "context": example["sql_context"],
#         "sql": example["sql"]
#     }

# new
def extract_fields_synthetic(example):
    return {
        "instruction": example["sql_prompt"],
        "input": example["sql_context"],
        "output": example["sql"]
    }
synthetic_extracted_dataset = dataset.map(extract_fields_synthetic, remove_columns=dataset['train'].column_names)

In [ ]:
# Split and shuffle datasets

import random 

synthetic_extracted_train_dataset = synthetic_extracted_dataset["train"]
synthetic_extracted_test_dataset = synthetic_extracted_dataset["test"]

# Shuffle the dataset
synthetic_extracted_dataset = synthetic_extracted_dataset.shuffle(seed=random.randint(10,99))
synthetic_extracted_dataset = synthetic_extracted_dataset.shuffle(seed=random.randint(10,99))

print(synthetic_extracted_train_dataset)
print(synthetic_extracted_test_dataset)

In [ ]:
# 'torch.cuda.is_bf16_supported()' is a function that checks if BF16 is supported on the current GPU. BF16 is a data type that uses 16 bits, like float16, but allocates more bits to the exponent, which can result in higher precision.
# 'attn_implementation' is a variable that will hold the type of attention implementation to be used.

if torch.cuda.is_bf16_supported():
  compute_dtype = torch.bfloat16
else:
  compute_dtype = torch.float16

attn_implementation = 'eager'
print(attn_implementation)
print(compute_dtype)

In [ ]:
#load tokenizr to prepare dataset

tokenizer = AutoTokenizer.from_pretrained(MODEL_ID, trust_remote_code=True)
tokenizer.padding_side = 'right' # to prevent warnings

In [ ]:
#Define message/prompt creation and formatting methods for the datasets
# #prompt will have our prompt/instruction
# #context will have out SQL context i.e table creation sql command

def create_message_column(row):
    message = []
    user = {
        "content": f"\n #prompt: {row['instruction']}\n #context: {row['input']}",
        "role": "user"
    }
    message.append(user)
    assistant = {
        "content": f"{row['output']}",
        "role": "assistant"
    }
    message.append(assistant)
    return {"message": message}

def format_dataset_with_chat_template(row):
    return {"text": tokenizer.apply_chat_template(row["message"], add_generation_prompt=False, tokenize=False)}

In [ ]:
# Apply create_message_column function
synthetic_extracted_train_dataset = synthetic_extracted_train_dataset.map(create_message_column)
synthetic_extracted_test_dataset = synthetic_extracted_test_dataset.map(create_message_column)

# Format dataset using 
synthetic_extracted_train_dataset = synthetic_extracted_train_dataset.map(format_dataset_with_chat_template)
synthetic_extracted_test_dataset = synthetic_extracted_test_dataset.map(format_dataset_with_chat_template)

# Output the results to verify
print(synthetic_extracted_train_dataset)
print(synthetic_extracted_test_dataset)

In [ ]:
#select subsets of datasets
# 75:25 dataset ratio

synthetic_extracted_train_dataset = synthetic_extracted_train_dataset.select(range(1000))
synthetic_extracted_test_dataset = synthetic_extracted_test_dataset.select(range(330))

In [ ]:
# MODEL_ID is a string that specifies the identifier of the pre-trained model that will be fine-tuned. 
MODEL_ID = "microsoft/Phi-3-mini-4k-instruct"

# NEW_MODEL_NAME is a string that specifies the name of the new model after fine-tuning.
NEW_MODEL_NAME = "sql-xp-phi-3-mini-4k"

In [ ]:
# 'hf_model_repo' is the identifier for the Hugging Face repository where you want to save the fine-tuned model.
hf_model_repo="spectrewolf8/"+NEW_MODEL_NAME

# Load Model on GPU 
# 'device_map' is set to {"": 0}, which means that the entire model will be loaded on GPU 0.
device_map = {"": 0}


# Bits and Bytes configuration for the model

# 'load_in_4bit' is a boolean that control if 4bit quantization should be loaded. In this case, it is set to True
# 'bnb_4bit_compute_dtype' is the data type that should be used for computations with the 4-bit base model. In this case, it is set to 'bfloat16'.
# 'bnb_4bit_quant_type' is the type of quantization that should be used for the 4-bit base model. In this case, it is set to 'nf4'.
# 'bnb_4bit_use_double_quant' is a boolean that controls whether nested quantization should be used for the 4-bit base model.

bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype="bfloat16",
        bnb_4bit_use_double_quant=True,
)

# LoRA configuration for the model

# 'lora_r' is the dimension of the LoRA attention.
# 'lora_alpha' is the alpha parameter for LoRA scaling.
# 'lora_dropout' is the dropout probability for LoRA layers.
# 'target_modules' is a list of the modules that should be targeted by LoRA.

lora_r = 16
lora_alpha = 16
lora_dropout = 0.05
target_modules= ['k_proj', 'q_proj', 'v_proj', 'o_proj', "gate_proj", "down_proj", "up_proj"]

# peft configuration for the model
peft_config = LoraConfig(
    lora_alpha = lora_alpha,
    lora_dropout = lora_dropout,
    r = lora_r,
    target_modules=target_modules
)


In [ ]:
# 'set_seed(1234)' sets the random seed for reproducibility.
set_seed(1234)

# username is a string that specifies the GitHub username of the person who is fine-tuning the model.
# license is a string that specifies the license under which the model is distributed. In this case, it's Apache License 2.0.

username = "spectrewolf8"
license = "apache-2.0"

# MAX_SEQ_LENGTH is an integer that specifies the maximum length of the sequences that the model will handle.
# num_train_epochs is an integer that specifies the number of times the training process will go through the entire dataset.
# learning_rate is a float that specifies the learning rate to be used during training.
# per_device_train_batch_size is an integer that specifies the number of samples to work through before updating the internal model parameters.
# gradient_accumulation_steps is an integer that specifies the number of steps to accumulate gradients before performing a backward/update pass.

MAX_SEQ_LENGTH = 2048
num_train_epochs = 1
learning_rate = 1.41e-5
per_device_train_batch_size = 4
gradient_accumulation_steps = 1

In [ ]:
# 'AutoTokenizer' is a class from the Hugging Face Transformers library that provides a tokenizer for a given pre-trained model.
# 'from_pretrained' is a method of the 'AutoTokenizer' class that loads a tokenizer from a pre-trained model.
# 'trust_remote_code=True' is a parameter that allows the execution of remote code when loading the tokenizer.
# 'add_eos_token=True' is a parameter that adds an end-of-sentence token to the tokenizer.
# 'use_fast=True' is a parameter that uses the fast version of the tokenizer, if available.
# 'tokenizer.pad_token = tokenizer.unk_token' sets the padding token of the tokenizer to be the same as the unknown token.
# 'tokenizer.pad_token_id = tokenizer.convert_tokens_to_ids(tokenizer.pad_token)' sets the ID of the padding token to be the same as the ID of the padding token.
# 'tokenizer.padding_side = 'left'' sets the side where padding will be added to be the left side.
# 'BitsAndBytesConfig' is a class that provides a configuration for quantization.
# 'bnb_config' is a variable that holds the configuration for quantization.

tokenizer = AutoTokenizer.from_pretrained(MODEL_ID, trust_remote_code=True, add_eos_token=True, use_fast=True)
tokenizer.pad_token = tokenizer.unk_token
tokenizer.pad_token_id = tokenizer.convert_tokens_to_ids(tokenizer.pad_token)
tokenizer.padding_side = 'left'

# 'AutoModelForCausalLM' is a class from the Hugging Face Transformers library that provides a model for causal language modeling.
# 'from_pretrained' is a method of the 'AutoModelForCausalLM' class that loads a model from a pre-trained model.
# 'torch_dtype=compute_dtype' is a parameter that sets the data type of the model to be the same as 'compute_dtype'.
# 'quantization_config=bnb_config' is a parameter that sets the configuration for quantization to be 'bnb_config'.
# 'device_map=device_map' is a parameter that sets the device map of the model to be 'device_map'.
# 'attn_implementation=attn_implementation' is a parameter that sets the type of attention implementation to be 'attn_implementation'.
# 'model = prepare_model_for_kbit_training(model)' prepares 'model' for k-bit training and assigns the result back to 'model'.

model = AutoModelForCausalLM.from_pretrained(
          MODEL_ID, torch_dtype=compute_dtype, trust_remote_code=True, quantization_config=bnb_config, device_map=device_map,
          attn_implementation=attn_implementation
)

model = prepare_model_for_kbit_training(model)
model.gradient_checkpointing_enable()

In [ ]:
# This block of code is used to initialize Weights & Biases (wandb) for experiment tracking.

# Retrieve the Weights & Biases API token from user secrets
wandb_token = user_secrets.get_secret("WANDB_TOKEN")

# Import the wandb library for experiment tracking
import wandb

# Log in to Weights & Biases using the retrieved API token
wandb.login(key=wandb_token)

# Initialize a new Weights & Biases run for tracking the experiment
run = wandb.init(
    project='Training and tuning Phi-3-mini-4k-instruct for SQL | kaggle-sql-xp-phi-3-mini-4k-instruct.ipynb', 
    job_type="training",  # Specify the type of job as training
    anonymous="allow"     # Allow anonymous logging if no user is logged in
)


In [ ]:
# 'TrainingArguments' is a class from the Hugging Face Transformers library that provides hyperparameters for training.
# 'output_dir="./results"' sets the directory where the training results (like checkpoints and logs) will be saved.
# 'num_train_epochs=1' sets the number of times the entire training dataset will be passed through the model.
# 'per_device_train_batch_size=4' sets the batch size for training on each device (e.g., GPU).
# 'gradient_accumulation_steps=1' sets the number of steps to accumulate gradients before performing a backward/update pass.
# 'optim="paged_adamw_32bit"' specifies the optimizer to use; in this case, "paged_adamw_32bit" is used.
# 'save_steps=25' specifies the number of steps before saving a checkpoint.
# 'logging_steps=10' specifies the number of steps before logging training metrics.
# 'learning_rate=2e-4' sets the learning rate for the optimizer.
# 'weight_decay=0.001' applies weight decay (L2 regularization) to prevent overfitting.
# 'fp16=False' specifies whether to use 16-bit (half-precision) floating point.
# 'bf16=False' specifies whether to use bfloat16 precision (an alternative to fp16).
# 'max_grad_norm=0.3' clips the gradient norm to prevent the exploding gradient problem.
# 'max_steps=-1' specifies the total number of training steps; -1 means no limit.
# 'warmup_ratio=0.03' sets the proportion of training steps to perform learning rate warmup.
# 'group_by_length=True' groups sequences of similar lengths together for efficient training.
# 'lr_scheduler_type="constant"' specifies the type of learning rate scheduler; in this case, it uses a constant learning rate.
# 'report_to="wandb"' specifies the reporting tool to use for logging; in this case, Weights and Biases (wandb) is used.

training_arguments = TrainingArguments(
    output_dir="./results",
    num_train_epochs=1,
    per_device_train_batch_size=4,
    gradient_accumulation_steps=1,
    optim="paged_adamw_32bit",
    save_steps=25,
    logging_steps=10,
    learning_rate=2e-4,
    weight_decay=0.001,
    fp16=False,
    bf16=False,
    max_grad_norm=0.3,
    max_steps=-1,
    warmup_ratio=0.03,
    group_by_length=True,
    lr_scheduler_type="constant",
    report_to="wandb"
)

In [ ]:
# 'SFTTrainer' is a class that provides a trainer for fine-tuning a model.
# 'trainer' is a variable that holds the trainer.
# 'model=model' is a parameter that sets the model to be trained to be 'model'.
# 'train_dataset=synthetic_extracted_train_dataset' is a parameter that sets the training dataset to be 'synthetic_extracted_train_dataset'.
# 'eval_dataset=synthetic_extracted_test_dataset' is a parameter that sets the evaluation dataset to be 'synthetic_extracted_test_dataset'.
# 'peft_config=peft_config' is a parameter that sets the configuration for the Lora layer to be 'peft_config'.
# 'dataset_text_field="text"' is a parameter that sets the field in the dataset that contains the text to be 'text'.
# 'max_seq_length=512' is a parameter that sets the maximum sequence length for the model to be 512.
# 'tokenizer=tokenizer' is a parameter that sets the tokenizer to be 'tokenizer'.
# 'args=args' is a parameter that sets the training arguments to be 'args'.
# This line of code is used to create a trainer for fine-tuning the model with the specified parameters.

trainer = SFTTrainer(
        model=model,
        train_dataset=synthetic_extracted_train_dataset,
        eval_dataset=synthetic_extracted_test_dataset,
        peft_config=peft_config,
        dataset_text_field="text",
        max_seq_length=512,
        tokenizer=tokenizer,
        args=training_arguments,
        callbacks=[CustomWandbCallback],
)

In [ ]:
# 'trainer.train()' is a method that starts the training of the model. It uses the training dataset, model, and training arguments that were specified when the trainer was created.

# train the model
trainer.train()

In [ ]:
# 'trainer.save_model()' is a method that saves the trained model to the local file system. The model will be saved in the output directory that was specified in the training arguments.
# This block of code is used to train the model and then save the trained model to the local file system.

# save model locally
trainer.save_model()
tokenizer.save_pretrained("./results")

In [ ]:
# trainer.save_model("./path_to_save_model")  # Save the model locally to specified directory
# tokenizer.save_pretrained("./path_to_save_model")  # Save the tokenizer to specified directory

In [ ]:
# Define the repository name on the Hugging Face Hub where the model, trainer, and tokenizer will be pushed.
hf_model_repo = "spectrewolf8/sql-xp-phi-3-mini-4k"

# Push the trainer to the Hugging Face Hub.
# This includes training arguments, optimizer states, and other relevant information.
trainer.push_to_hub(hf_model_repo)

# Push the model to the Hugging Face Hub.
# This saves the model weights and configuration to the specified repository.
trainer.model.push_to_hub(hf_model_repo)

# Push the tokenizer to the Hugging Face Hub.
# This saves the tokenizer configuration and vocab files to the specified repository.
tokenizer.push_to_hub(hf_model_repo)

# Model stats

In [ ]:
# Finish the Weights & Biases (wandb) run.
# This finalizes the current experiment run, ensuring all data is uploaded and the run is properly closed.
wandb.finish()

# Set the 'use_cache' configuration option of the model to True.
# This enables caching of the computation results during inference, which can speed up the model's performance.
model.config.use_cache = True

# Set the model to evaluation mode.
# This changes the model's behavior to inference mode, disabling features like dropout that are only used during training.
model.eval()

# Testing model

In [ ]:
# Create a text generation pipeline using the specified model and tokenizer.
# The 'pipeline' function sets up a ready-to-use text generation pipeline, combining the model and tokenizer.
pipe = pipeline("text-generation", model=model, tokenizer=tokenizer)

## example-1

In [ ]:
# Define the input phrase which represents the user's request or query.
input_phrase = """
insert 5 values
"""

# Define the context phrase which provides the SQL table schema relevant to the input phrase.
context_phrase = """
CREATE TABLE tasks (
    id INT AUTO_INCREMENT PRIMARY KEY,
    name VARCHAR(100) NOT NULL,
    task_name VARCHAR(100) NOT NULL,
    userid INT NOT NULL,
    date DATE NOT NULL,
    FOREIGN KEY (userid) REFERENCES users(id)
);
"""

# Create a prompt by applying a chat template to the input and context phrases using the tokenizer.
# The 'apply_chat_template' method formats the input as a chat message, making it suitable for text generation.
# 'tokenize=False' indicates that the input should not be tokenized yet.
# 'add_generation_prompt=True' adds a prompt for text generation.
prompt = pipe.tokenizer.apply_chat_template(
    [{"role": "user", "content": f"\n {input_phrase} Input:{context_phrase}"}],
    tokenize=False,
    add_generation_prompt=True
)

# Generate text using the pipeline with the specified parameters.
# 'max_new_tokens=256' sets the maximum number of new tokens to generate.
# 'do_sample=True' enables sampling for text generation.
# 'num_beams=1' specifies the number of beams for beam search (1 means no beam search).
# 'temperature=0.3' controls the randomness of predictions by scaling the logits before applying softmax.
# 'top_k=50' considers only the top 50 token predictions for sampling.
# 'top_p=0.95' enables nucleus sampling, considering tokens that have a cumulative probability of 0.95.
# 'max_time=180' sets the maximum generation time to 180 seconds.
outputs = pipe(
    prompt,
    max_new_tokens=256,
    do_sample=True,
    num_beams=1,
    temperature=0.3,
    top_k=50,
    top_p=0.95,
    max_time=180
)

# Print the generated text by stripping out the prompt portion and displaying only the new generated content.
print(outputs[0]['generated_text'][len(prompt):].strip())

## example-2

In [ ]:
# Define a list of input phrases representing various SQL operations.
input_phrases = [
    "insert 5 values",
    "select all records",
    "update record with id 3",
    "delete all records where task_name is 'coding'",
    "add a new column 'status' to the table",
    "find all tasks with userid 2",
    "count the number of tasks per user",
    "list all tasks sorted by date",
    "join tasks with users",
    "find the average number of tasks per user"
]

# Define a list of context phrases which provide the SQL table schema.
# The same context is used for all input phrases in this example.
context_phrases = [
    """
    CREATE TABLE tasks (
        id INT AUTO_INCREMENT PRIMARY KEY,
        name VARCHAR(100) NOT NULL,
        task_name VARCHAR(100) NOT NULL,
        userid INT NOT NULL,
        date DATE NOT NULL,
        FOREIGN KEY (userid) REFERENCES users(id)
    );
    """
] * len(input_phrases)  # Repeat the same context for all input phrases.

# Apply the chat template to create prompts by combining input and context phrases.
# The 'apply_chat_template' method formats each input phrase with its corresponding context phrase.
# f"\n #prompt: {input_phrase}\n #context: {context_phrase}"

prompts = [pipe.tokenizer.apply_chat_template(
    [{"role": "user", "content": f"\n #prompt: {input_phrase}\n #context: {context_phrase}"}], 
    tokenize=False, 
    add_generation_prompt=True
) for input_phrase, context_phrase in zip(input_phrases, context_phrases)]

# Generate SQL queries using the text generation pipeline with specified parameters.
# Each prompt is passed through the pipeline to generate the corresponding SQL query.
outputs = [pipe(
    prompt, 
    max_new_tokens=256, 
    do_sample=True, 
    num_beams=1, 
    temperature=0.3, 
    top_k=50, 
    top_p=0.95, 
    max_time=180
) for prompt in prompts]

# Print the results of the generated SQL queries.
# For each generated output, strip out the prompt portion and display only the new generated content.
for i, output in enumerate(outputs):
    generated_text = output[0]['generated_text'][len(prompts[i]):].strip()
    print(f"Prompt {i+1}:")
    print(generated_text)
    print("\n")

# Loading model from hugging face

In [ ]:
# Import necessary libraries
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, set_seed

# Set the seed for the random number generator to ensure reproducibility
set_seed(1234)

# Define the repository name for the Hugging Face model
# 'hf_model_repo' is a variable that holds the repository name for the Hugging Face model
# 'username/modelname' is the repository name, where 'username' is the username of the repository owner
# and 'modelname' is the name of the model
hf_model_repo = "spectrewolf8/sql-xp-phi-3-mini-4k"

# Retrieve the device mapping and computation data type
# 'device_map' is a variable that holds the mapping of the devices that are used for computation
# 'compute_dtype' is a variable that holds the data type that is used for computation

# device_map = {"": 0}
# compute_dtype = torch.bfloat16 or torch.float16
device_map, compute_dtype

# Load a pre-trained tokenizer from the Hugging Face Model Hub
# 'tokenizer' is the variable that holds the tokenizer
# 'trust_remote_code=True' allows the execution of code from the model file
tokenizer = AutoTokenizer.from_pretrained(hf_model_repo, trust_remote_code=True)

# Load a pre-trained model for causal language modeling from the Hugging Face Model Hub
# 'model' is the variable that holds the model
# 'trust_remote_code=True' allows the execution of code from the model file
# 'torch_dtype=compute_dtype' sets the data type for the PyTorch tensors
# 'device_map=device_map' sets the device mapping
model = AutoModelForCausalLM.from_pretrained(hf_model_repo, trust_remote_code=True, torch_dtype=compute_dtype, device_map=device_map)


In [ ]:
pipe = pipeline("text-generation", model=model, tokenizer=tokenizer)

### Rest of the steps from here are the same as example-1 and example-2

In [ ]:
# Define the context and input phrase
context_phrase = """
CREATE TABLE users (
    id INT AUTO_INCREMENT PRIMARY KEY,
    username VARCHAR(50) NOT NULL,
    email VARCHAR(100) NOT NULL,
    created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
);

CREATE TABLE projects (
    id INT AUTO_INCREMENT PRIMARY KEY,
    project_name VARCHAR(100) NOT NULL,
    description TEXT,
    start_date DATE NOT NULL,
    end_date DATE,
    user_id INT,
    FOREIGN KEY (user_id) REFERENCES users(id)
);

CREATE TABLE tasks (
    id INT AUTO_INCREMENT PRIMARY KEY,
    task_name VARCHAR(100) NOT NULL,
    status VARCHAR(20) CHECK (status IN ('pending', 'in_progress', 'completed')),
    priority INT CHECK (priority BETWEEN 1 AND 5),
    project_id INT,
    assigned_to INT,
    due_date DATE,
    FOREIGN KEY (project_id) REFERENCES projects(id),
    FOREIGN KEY (assigned_to) REFERENCES users(id)
);

CREATE TABLE comments (
    id INT AUTO_INCREMENT PRIMARY KEY,
    task_id INT,
    user_id INT,
    comment_text TEXT NOT NULL,
    created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
    FOREIGN KEY (task_id) REFERENCES tasks(id),
    FOREIGN KEY (user_id) REFERENCES users(id)
);
"""

input_phrase = """
Update the status of tasks to 'completed' for all tasks that have passed their due date. Also, update the end date of the corresponding projects to the current date if all tasks in the project are completed.
"""

# Apply the chat template to create the prompt
prompt = pipe.tokenizer.apply_chat_template([{"role": "user", "content": f"\n #prompt: {input_phrase}\n #context: {context_phrase}"}], tokenize=False, add_generation_prompt=True)

# Generate SQL query
outputs = pipe(prompt, max_new_tokens=256, do_sample=True, num_beams=1, temperature=0.3, top_k=50, top_p=0.95, max_time=180)

# Print the result
generated_text = outputs[0]['generated_text'][len(prompt):].strip()
print(f"Generated SQL Query:\n{generated_text}")

In [ ]:
print(prompt)